In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai_key = os.getenv('OPENAI_API_KEY')

# Print the OpenAI API key
#print(openai_key)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "./data/Lietuvos istorija - Visuotinė lietuvių enciklopedija.pdf"
)

loader = PyPDFLoader(file_path)

visuotine_documents = loader.load()
#from rich import print
#print(visuotine_documents)     

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "./data/Lietuvos_istorija.pdf"
)

loader = PyPDFLoader(file_path)

vikipedija_documents = loader.load()
#from rich import print
#print(vikipedija_documents)

In [ ]:
merged_documents = visuotine_documents + vikipedija_documents

#print(merged_documents)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(merged_documents)

print(f"Split blog post into {len(all_splits)} sub-documents.")

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=openai_key)

from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

document_ids = vector_store.add_documents(documents=all_splits)

In [11]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(čia įrašykite kontekstą)", "question": "(čia įrašykite klausimą)"}
).to_messages()

assert len(example_messages) == 1
#print(example_messages[0].content)

c:\Users\gaidy\Documents\genai\genai\.venv\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
from langchain.llms import OpenAI

llm = OpenAI(api_key=openai_key)

question = "surask žymiausius mūšius lietuvoje."

retrieved_docs = vector_store.similarity_search(question)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
promptAnswer = prompt.invoke({"question": question, "context": docs_content})
answer = llm.invoke(promptAnswer)
print(answer)

Žalgirio mūšis yra vienas iš žymiausių mūšių Lietuvos istorijoje. Jis vyko 1410 m. liepos 15 d. ir buvo kovojama 
tarp Lietuvos ir Lenkijos karių bei Kryžiuočių ir Livonijos ordinų. Mūšis baigėsi Lietuvos ir Lenkijos karių 
pergalėmis. Mūšyje dalyvavo ir Lietuvos didysis kunigaikštis Vytautas.